# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [ ]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [ ]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [ ]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [ ]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
import nltk
from nltk.corpus import brown
import numpy as np


nltk.download('brown')
np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
# choose words that are entirely comprised of alphabetic characters. 
# lowercase the words and get rid of duplicates
words = set([word.lower() for word in brown.words() if word.isalpha()])
words = list(words)

# shuffle the words
np.random.shuffle(words)

# the test set contains 1000 word types,
test_set = words[:1000]
training_set = words[1000:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [ ]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [ ]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (0.5 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [ ]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    
    # obtain the letters that are not guessed
    letters = set('abcdefghijklmnopqrstuvwxyz').difference(guessed)
    guess = np.random.choice(list(letters))
    return guess
    
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = acclaim
Number of mistakes made by the random guesser = 16

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.66


<b>For your testing:</b>

In [ ]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method (use the second answer space). Note that it takes `unigram_counts` as an additional argument.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
unigram_counts = None

###
# Your answer BEGINS HERE
###
import collections
unigram_counts = collections.defaultdict(int) 
# collect the frequencies of characters and store them in the dictionary
for word in training_set:
    for letter in word:
        unigram_counts[letter] += 1
###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    
    letters = set(unigram_counts.keys()).difference(guessed)
    # select a letter from the unguessed letters that has the highest counts among all letters
    guess = max(letters, key = lambda x: unigram_counts[x])
    return guess
    
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.048


<b>For your testing:</b>

In [ ]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###

# collect the frequencies of characters conditioned on the word length 
# store them in the format of {len: {'a': count, 'b': count, ...}}
unigram_counts_by_length = collections.defaultdict(int) 
for word in training_set:
    length = len(word)
    
    # initialize the dictionary when seeing a new letter
    if length not in unigram_counts_by_length:
       unigram_counts_by_length[length] = collections.defaultdict(int)
    
    for letter in word:
        unigram_counts_by_length[length][letter] += 1
###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###

    mask_len = len(mask)
    letters = set(unigram_counts.keys()).difference(guessed)

    # the length is not in the training set
    if mask_len not in unigram_counts_by_length: 
        dict_to_use = unigram_counts # fall back to unigram_counts
    else:
        dict_to_use = unigram_counts_by_length[mask_len]
        
    # the length is in the training set but we guessed all seen letters
    if not letters: 
        return max(unigram_counts.keys(), key = lambda x: unigram_counts[x]) # fall back to unigram_counts
        
    # select a letter from the remaining letters
    guess = max(letters, key = lambda x: dict_to_use[x])
    return guess
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.09


<b>For your testing:</b>

In [ ]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [ ]:
bigram_counts = None

###
# Your answer BEGINS HERE
###

bigram_counts = collections.defaultdict(collections.Counter)
# collect bigram counts
for word in training_set:
    # handling the start of each word by padding with $. 
    word = '$' + word
    bigram_list = zip(word[:-1], word[1:])
    # iterate over bigrams
    for bigram in bigram_list:
        first, second = bigram
        bigram_counts[first][second] += 1 
        
# function to calculate the probability of seeing all characters from a to z based on a given letter      
def compute_probs(counts):
    probs = collections.defaultdict(int)
    for k in counts.keys():
        probs[k] = counts[k] / sum(list(counts.values()))
    return probs
        
# calculate bigram probs based on bigram counts
bigram_probs = collections.defaultdict(int)
for k in bigram_counts.keys():
    bigram_probs[k] = compute_probs(bigram_counts[k])
    
# calculate unigram probs based on unigram counts
unigram_probs = compute_probs(unigram_counts)

###
# Your answer ENDS HERE
###
def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_probs=unigram_probs, bigram_probs=bigram_probs): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    
    # sum of probability distribution over all alphabets for all blanks
    probs = collections.defaultdict(int)
    left = 0            # left index
    right = 1           # right index
    mask = ['$'] + mask # pad the mask 
    
    while right < len(mask):
        first_char = mask[left]
        second_char = mask[right]

        # guess the second character based on the known first character
        if first_char != '_' and second_char == '_':
            letters = set(bigram_counts[first_char].keys()).difference(guessed)
            probs_to_use = bigram_probs[first_char]
            
            # fall back to vanilla unigram counts if running out of seen letters in the bigram counts
            if not letters:
                letters = set(unigram_counts.keys()).difference(guessed)
                probs_to_use = unigram_probs
            
            # compute the probability of the remaining letters
            for k in letters:
                probs[k] += probs_to_use[k]
        
        # both first and second characters are unknown, so fall back to vanilla unigram counts        
        elif first_char == '_' and second_char == '_':
            letters = set(unigram_counts.keys()).difference(guessed)
            probs_to_use = unigram_probs
            for k in letters:
                probs[k] += probs_to_use[k]
                
        left += 1
        right += 1
    
    return max(probs, key=probs.get)
    ###
    # Your answer ENDS HERE
    ###
 
result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.547


### Question 6 (2.0 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 2.0 mark if x < 6.0
* 1.5 mark if 6.0 <= x < 6.5
* 1.0 mark if 6.5 <= x < 7.0
* 0.5 mark if 7.0 <= x < 8.0
* 0.0 mark if x >= 8.0

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its _average_ performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 2 minutes on Colab (free account) with GPU; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [ ]:
###
# Your answer BEGINS HERE
###
alphabet = 'abcdefghijklmnopqrstuvwxyz'
k = 0.005

# compute the probability with add-k smoothing
def compute_probs_smoothing(counts, letters, k):
    probs = collections.defaultdict(int)
    for l in letters:
        probs[l] = (counts[l]+k) / (sum(list(counts.values())) + k*len(alphabet))
    return probs  

# collect normal bigarm counts
bigram_counts_nor = collections.defaultdict(collections.Counter)
for word in training_set:
    word = '$' + word + '.'
    bigram_list = zip(word[:-1], word[1:])
    # iterate over bigrams
    for bigram in bigram_list:
        first, second = bigram
        bigram_counts_nor[first][second] += 1 # a dict of dict
  
# collect reversed bigram count 
bigram_counts_rev = collections.defaultdict(collections.Counter)
for word in training_set:
    word = '$' + word + '.' # pad the end of a word with .
    bigram_list = zip(word[:-1], word[1:])
    # iterate over bigrams
    for bigram in bigram_list:
        first, second = bigram
        bigram_counts_rev[second][first] += 1 # a dict of dict
        
# collect CBOW count (window size = 3)
cbow_counts = collections.defaultdict(collections.Counter)
for word in training_set:
    word = list('$' + word + '.')
    for i in range(1, len(word) - 1):
        cbow_counts[(word[i-1], word[i+1])][word[i]] += 1

# collect CBOW count (window size = 5)
cbow_counts_long = collections.defaultdict(collections.Counter)
for word in training_set:
    word = list('$$' + word + '..')
    for i in range(2, len(word) - 2):
        cbow_counts_long[(word[i-2], word[i-1], word[i+1], word[i+2])][word[i]] += 1


def my_amazing_ai_guesser(mask, guessed, unigram_counts=unigram_counts, 
                          bigram_counts_nor=bigram_counts_nor, bigram_counts_rev=bigram_counts_rev, 
                          cbow_counts=cbow_counts, cbow_counts_long=cbow_counts_long): 
    
    # probability distribution over all unguessed letters
    probs = collections.defaultdict(int)
    # all unguessed letters
    letters = set(alphabet).difference(guessed)
    # pad the mask 
    
    mask = ['$'] + ['$'] + mask + ['.'] + ['.']
    
    for i in range(1, len(mask)-1):
        # skip the characters that have been guessed 
        if mask[i] != '_':
            continue

        left_char = mask[i-1]
        right_char = mask[i+1]
        neighbor = False # whether the missing character has at least one known neighbor
        
        # CBOW with a window size of 3
        if left_char != '_' and right_char != '_':
            neighbor = True   
            prob = compute_probs_smoothing(cbow_counts[(left_char, right_char)], letters, k) 
            # CBOW with a window size of 5 
            if i >= 2 and i < len(mask)-2 and mask[i-2] != '_' and mask[i+2] != '_':
                prob = compute_probs_smoothing(cbow_counts_long[(mask[i-2], left_char, right_char, mask[i+2])], letters, k)          
        # bigram normal
        if left_char != '_' and right_char == '_':
            neighbor = True  
            prob = compute_probs_smoothing(bigram_counts_nor[left_char], letters, k) 
        # bigram reversed
        if left_char == '_' and right_char != '_':
            neighbor = True  
            prob = compute_probs_smoothing(bigram_counts_rev[right_char], letters, k)
        # unigram
        if not neighbor:
            # fall back to unigram if seeing three '_' consecutively    
            prob = compute_probs_smoothing(unigram_counts, letters, k)

        for key in prob.keys():
            probs[key] += prob[key]   

    return max(probs, key=probs.get)
###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.574


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

I implemented several techniques to improve the hangman AI's performance. 

- **Add-k smoothing**: I applied add-k smoothing when computing bigram and CBOW probabilities. Only the unigram model sees all characters in training. Other models like bigram or CBOW models don't know how to handle unseen character combinations like `"xr"` or `"zx"` when testing. Therefore, smoothing is essential. I experimented with k values from 1 to 0.005 and found smaller values yielded marginally better results generally.

- **Bidirectional bigram**: To improve contextual understanding, I incorporated a bidirectional bigram model and pad the word with a period `.` at the end. Unlike the normal bigram which looks left-to-right, the reversed bigram tracks characters that precede a given letter. For instance, after seeing `"ea"`, `"ra"`, `"ta"` and `"ta"`, it stores the counts as `{'a': {'e': 1, 'r': 1, 't': 2}}`. This boosted performance by about 0.5 fewer mistakes on average.

- **CBOW**: Unlike bigram models, CBOW takes into account both side of the middle characters at the same time. Initially, I used a window of size 3 and let's say, the model sees `'cat'` during training, it records `{('c','t'): {'a':1}}`. This helped the model in the later stages of guessing, reducing errors from 8.3 to around 7.7. Extending the window to size 5 further improved accuracy to nearly 7.5. However, a window of 7 worsened performance to roughly 8.9 mistakes, probably because such wide contexts were too specific and didn’t generalize well. Moreover, Some words are shorter than 7 letters after padding, and even longer ones might not provide consistent patterns across training and test sets. Note that if a word satisfies the conditions of CBOW with both window sizes, the model calculates probabilities twice using different window sizes. I made this design decision because there are lots of unseen 5-grams, so the probabilities computed according to CBOW with a window size of 5 may not be beneficial sometimes, requiring the assistance of CBOW with a smaller window size.